In [ ]:
! pip install pyspark

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 317.3/317.3 MB 1.5 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wheel for pyspark: filename=pyspark-3.5.2-py2.py3-none-any.whl size=317812365 sha256=c98db1fe8d11043da8a604b93163426f43bc4f0ab28e5716620fde7842b3f97c
  Stored in directory: /root/.cache/pip/wheels/34/34/bd/03944534c44b677cd5859f248090daa9fb27b3c8f8e5f49574
Successfully built pyspark


In [ ]:
from pyspark.sql import SparkSession
spark = SparkSession.builder.appName("MovieDataTransformations").getOrCreate()
movies_df = spark.read.csv("/content/sample_data/movie_data.csv", header=True, inferSchema=True)
movies_df.show()


+--------+-----------------+---------+------+----------+----------+
|movie_id|            title|    genre|rating|box_office|      date|
+--------+-----------------+---------+------+----------+----------+
|       1|        Inception|   Sci-Fi|   8.8| 830000000|2010-07-16|
|       2|  The Dark Knight|   Action|   9.0|1004000000|2008-07-18|
|       3|     Interstellar|   Sci-Fi|   8.6| 677000000|2014-11-07|
|       4|Avengers: Endgame|   Action|   8.4|2797000000|2019-04-26|
|       5|    The Lion King|Animation|   8.5|1657000000|1994-06-15|
|       6|      Toy Story 4|Animation|   7.8|1073000000|2019-06-21|
|       7|        Frozen II|Animation|   7.0|1450000000|2019-11-22|
|       8|            Joker|    Drama|   8.5|1074000000|2019-10-04|
|       9|         Parasite|    Drama|   8.6| 258000000|2019-05-30|
+--------+-----------------+---------+------+----------+----------+



In [ ]:
sci_fi_movies = movies_df.filter(movies_df.genre == "Sci-Fi")
sci_fi_movies.show()

+--------+------------+------+------+----------+----------+
|movie_id|       title| genre|rating|box_office|      date|
+--------+------------+------+------+----------+----------+
|       1|   Inception|Sci-Fi|   8.8| 830000000|2010-07-16|
|       3|Interstellar|Sci-Fi|   8.6| 677000000|2014-11-07|
+--------+------------+------+------+----------+----------+



In [ ]:
top_rated_movies = movies_df.orderBy(movies_df.rating.desc()).limit(3)
top_rated_movies.show()

+--------+---------------+------+------+----------+----------+
|movie_id|          title| genre|rating|box_office|      date|
+--------+---------------+------+------+----------+----------+
|       2|The Dark Knight|Action|   9.0|1004000000|2008-07-18|
|       1|      Inception|Sci-Fi|   8.8| 830000000|2010-07-16|
|       3|   Interstellar|Sci-Fi|   8.6| 677000000|2014-11-07|
+--------+---------------+------+------+----------+----------+



In [ ]:
from pyspark.sql.functions import year
movies_after_2010 = movies_df.filter(year(movies_df.date) > 2010)
movies_after_2010.show()

+--------+-----------------+---------+------+----------+----------+
|movie_id|            title|    genre|rating|box_office|      date|
+--------+-----------------+---------+------+----------+----------+
|       3|     Interstellar|   Sci-Fi|   8.6| 677000000|2014-11-07|
|       4|Avengers: Endgame|   Action|   8.4|2797000000|2019-04-26|
|       6|      Toy Story 4|Animation|   7.8|1073000000|2019-06-21|
|       7|        Frozen II|Animation|   7.0|1450000000|2019-11-22|
|       8|            Joker|    Drama|   8.5|1074000000|2019-10-04|
|       9|         Parasite|    Drama|   8.6| 258000000|2019-05-30|
+--------+-----------------+---------+------+----------+----------+



In [ ]:
avg_box_office_by_genre = movies_df.groupBy("genre").avg("box_office").alias("average_box_office")
avg_box_office_by_genre.show()

+---------+--------------------+
|    genre|     avg(box_office)|
+---------+--------------------+
|    Drama|              6.66E8|
|Animation|1.3933333333333333E9|
|   Action|            1.9005E9|
|   Sci-Fi|             7.535E8|
+---------+--------------------+



In [ ]:
movies_df = movies_df.withColumn("box_office_billion", movies_df.box_office / 1e9)
movies_df.show()

+--------+-----------------+---------+------+----------+----------+------------------+
|movie_id|            title|    genre|rating|box_office|      date|box_office_billion|
+--------+-----------------+---------+------+----------+----------+------------------+
|       1|        Inception|   Sci-Fi|   8.8| 830000000|2010-07-16|              0.83|
|       2|  The Dark Knight|   Action|   9.0|1004000000|2008-07-18|             1.004|
|       3|     Interstellar|   Sci-Fi|   8.6| 677000000|2014-11-07|             0.677|
|       4|Avengers: Endgame|   Action|   8.4|2797000000|2019-04-26|             2.797|
|       5|    The Lion King|Animation|   8.5|1657000000|1994-06-15|             1.657|
|       6|      Toy Story 4|Animation|   7.8|1073000000|2019-06-21|             1.073|
|       7|        Frozen II|Animation|   7.0|1450000000|2019-11-22|              1.45|
|       8|            Joker|    Drama|   8.5|1074000000|2019-10-04|             1.074|
|       9|         Parasite|    Drama|   8.

In [ ]:
sorted_by_box_office = movies_df.orderBy(movies_df.box_office.desc())
sorted_by_box_office.show()

+--------+-----------------+---------+------+----------+----------+------------------+
|movie_id|            title|    genre|rating|box_office|      date|box_office_billion|
+--------+-----------------+---------+------+----------+----------+------------------+
|       4|Avengers: Endgame|   Action|   8.4|2797000000|2019-04-26|             2.797|
|       5|    The Lion King|Animation|   8.5|1657000000|1994-06-15|             1.657|
|       7|        Frozen II|Animation|   7.0|1450000000|2019-11-22|              1.45|
|       8|            Joker|    Drama|   8.5|1074000000|2019-10-04|             1.074|
|       6|      Toy Story 4|Animation|   7.8|1073000000|2019-06-21|             1.073|
|       2|  The Dark Knight|   Action|   9.0|1004000000|2008-07-18|             1.004|
|       1|        Inception|   Sci-Fi|   8.8| 830000000|2010-07-16|              0.83|
|       3|     Interstellar|   Sci-Fi|   8.6| 677000000|2014-11-07|             0.677|
|       9|         Parasite|    Drama|   8.

In [ ]:
movies_count_by_genre = movies_df.groupBy("genre").count()
movies_count_by_genre.show()

+---------+-----+
|    genre|count|
+---------+-----+
|    Drama|    2|
|Animation|    3|
|   Action|    2|
|   Sci-Fi|    2|
+---------+-----+

